# The Widely-Applicable Information Criterion (WAIC)

In [77]:
import pandas as pd
import pymc3 as pm
import numpy as np

from scipy.stats import norm
from scipy.special import logsumexp

import matplotlib
%matplotlib inline

The WAIC is defined as:


In [14]:
# read the `cars` dataset, which is standard in R
cars = pd.read_csv('https://raw.githubusercontent.com/vincentarelbundock/Rdatasets/master/csv/datasets/cars.csv', 
                   delimiter=',',
                   index_col=0)

cars.head()

,speed,dist
1,4,2
2,4,10
3,7,4
4,7,22
5,8,16


In [101]:
model = pm.Model()
n_samples = 1000

with model:

    # Priors for unknown model parameters
    a = pm.Normal('a', mu=0, sd=100)
    b = pm.Normal('b', mu=0, sd=10)
    mu = a + b * cars['speed']

    sigma = pm.Uniform('sigma', lower=0, upper=30)
    
    # Likelihood (sampling distribution) of observations
    distance = pm.Normal('distance', mu=mu, sd=sigma, observed=cars['dist'])
    
    trace = pm.sample(n_samples, tune=1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
100%|██████████| 2000/2000 [00:04<00:00, 481.05it/s]


We now want to find the (log) probability density (logpdf) of each observation, for each point in our posterior estimates.

In [118]:
cars['loglikelihood'] = np.zeros(len(cars))

ll_var = 0
m_loglike = 0

# for each observation i:
for i, (distance, speed, ll) in enumerate(zip(cars['dist'], cars['speed'], cars['loglikelihood'])):
        
    mu = trace['a'] + speed * trace['b']
    sigma = trace['sigma']
    
    #calculate the log-likelihood
    loglike = norm.logpdf(distance, loc=mu, scale=sigma)
#     m_loglike += np.log(np.mean(np.exp(loglike)))
    #  m_loglike += logsumexp(loglike) - np.log(n_samples)
    
    like = norm.pdf(distance, loc=mu, scale=sigma)
    m_loglike += np.log(np.mean(like))

        
    
    ll_var += np.var(loglike)
    
-2*(m_loglike - ll_var)

419.68115311406956

## A comment about precision
Since logpdf is a small quantity, calculating the mean of the log likelihood is less accurate:
$$ mean(LogLiklihood) = \frac{\sum_{n}loglike_n}{n} $$
We can use the `scipy.special.logsumexp` function, which sums the exponentials of the log-likelihood (which is just the likelihood) in this way:

$$\frac{\sum_{n}loglike_n}{n} = \log (\frac{\sum_{n}\exp(loglike_n)}{n}) = \log (\sum_{n}\exp(loglike_n)) - \log(n))$$